In [2]:
#! /usr/bin/env python

from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local[2]"))

import re

In [5]:
def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
    except ValueError as e:
        return []
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    word_pair = list(zip(words[:-1], words[1:]))
    word_pair = [word2.lower() for word1, word2 in word_pair if word1.lower() == 'narodnaya']
    return word_pair

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).flatMap(parse_article).filter(lambda x: x != []).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).sortByKey()  
result = wiki.map(lambda x: ('narodnaya_' + x[0], x[1])).collect()

for pair_count in result:
    print "%s\t%d" % (pair_count)


narodnaya_gazeta	1
narodnaya_volya	9
